# 第5单元: 使用PyTorch编写你的第一个深度强化学习算法: Reinforce. 并测试它的鲁棒性 💪

在这份笔记中, 你将从头编写你的第一个深度强化学习算法: Reinforce(也称为蒙特卡洛策略梯度).

Reinforce是一种**基于策略的方法**: 一种**尝试直接优化策略而不是使用动作价值函数**的深度强化学习算法. 更准确的说, Reinforce是策略梯度方法, 是基于策略方法的子类, 旨在**通过使用梯度提升(Gradient Ascent)估计最优策略的权重来直接优化策略.**

为了测试鲁棒性, 我们将在3个不同的简单环境进行训练:
* Cartpole-v1
* PixelCopterEnv
* PongEnv

❓如果你有任何问题, 请在discord的#study-group-unit5频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境:

* [CartPole-v1](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)
* [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)
* [Pong](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pong.html)

⬇️ 这是**你将在几分钟内实现的目标**的示例. ⬇️

![Sans titre.gif](./assets/Sans%20titre.gif)

## 这份笔记的目标🏆

在这份笔记学习结束后, 你将:

* 能够使用**PyTorch从头编写Reinforce算法.**
* 能够使用**简单的环境测试你的智能体的鲁棒性.**
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub.**

## 这份笔记来自深度强化学习课程
![Deep Reinforcement Learning Course.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践.**
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ **在独特的环境中训练智能体.**

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第5单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit5/README.md)

🔲 📚 通过阅读 👉 https://huggingface.co/blog/deep-rl-pg **学习策略梯度**

### 第0步: 设置GPU 💪

* 为了**更快的训练智能体, 我们将使用GPU,** 选择`修改 > 笔记本设置`
![image.png](./assets/image.png)

* `硬件加速器 > GPU`

![image.png](./assets/image1.png)

在笔记中, 我们需要生成一个回放视频. 因此在Colab(或你本地的jupyter)中, **我们需要一个虚拟屏幕能渲染环境**(记录视频帧).

下面的单元格将安装虚拟屏幕库并创建和运行虚拟屏幕. 🖥

In [ ]:
!apt install gitlfs ffmpeg
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

In [ ]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=False, size=(1400, 900))
virtual_display.start()

### 第1步: 安装依赖项 🔽
第一步是安装多个依赖项:
* `gym`: 包含Cartpole-v1, PixelCopter和Pong环境.
* `gym-games`: 使用PyGame制作的gym环境.
* `huggingface_hub`: 🤗 是一个任何人都可以分享和探索模型和数据集的地方. 它有版本控制, 评估, 可视化和其他功能, 可以允许你简单地与他人协作.

你可以在这里看到全部可用的Reinforce模型. 👉 https://huggingface.co/models?other=reinforce

你可以在这看到全部可用的深度强化学习模型. 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning

In [ ]:
!pip install gym
!pip install git+https://github.com/ntasfi/PyGame-Learning-Environment.git
!pip install git+https://github.com/qlan3/gym-games.git
!pip install huggingface_hub
!pip install pyglet  # 如果你使用IDE, 则不需要这些步骤.

### 第2步: 导入包 📦

除了安装的库, 我们还使用:
* `imageio`: 生成回放视频.

In [ ]:
from collections import deque

import gym
import gym_pygame

import imageio
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.distributions import Categorical

* 让我们检查我们是否有GPU.
* 如果有, 你应该看到`cuda:0`或者`mps`.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
print(device)

我们现在已经准备好实现我们的Reinforce算法 🔥

## 第1个智能体: 玩CartPole-v1 🤖️

### 第3步: 创建并理解CartPole环境
#### [环境 🎮 ](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)

![cartpole.jpg](./assets/cartpole.jpg)

### 为什么我们使用像CartPole-v1这样的简单环境?
正如[强化学习技巧](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)所解释的, 当你从头开始实现智能体时, 你需要**确保它可以正确工作, 并在深入之前找到简单环境中的bug.** 因为在简单环境中更容易找到bug.
> 尝试在玩具问题上有一些"生命迹象".

> 通过在越来越复杂的环境上运行来验证实现(你可以和RL zoo的结果进行比较). 你通常需要为该步骤运行超参数优化.

---

#### CartPole-v1环境

> 一根杆通过一个未驱动的接头连接到推车上, 该推车沿着无摩擦的轨道移动. 摆锤直立放置在推车上, 目标是通过在推车上施加左右方向的力, 使杆保持平衡.

所以, 我们从CartPole-v1开始. 目标是向左或者向右推动车辆, **使杆保持平衡.**

如果出现以下的情况, 则该局游戏结束:
* 杆的倾角超过±12˚
* 推车的位置变化超过±2.4
* 每轮游戏超过500步

杆保持平衡的每个时间步, 我们都会得到+1的奖励分数 💰.

In [ ]:
env_id = 'CartPole-v1'
# 创建环境.
env = gym.make(env_id)

# 创建评估环境.
eval_env = gym.make(env_id)

# 获取状态空间和动作空间的大小.
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print('_' * 5 + '可观察的环境' + '_' * 5, end='\n\n')
print('可观察的环境向量的形状', s_size)
print('随机采样环境', env.observation_space.sample())  # 获得一个随机的可观察环境空间.

In [ ]:
print('_' * 5 + '动作空间' + '_' * 5, end='\n\n')
print('动作的总数', a_size)
print('随机动作', env.action_space.sample())  # 获得一个随机的动作.

### 第4步: 让我们构建Reinforce架构
此实现基于两个实现:
* [PyTorch官方强化学习示例](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
* [Udacity的Reinforce算法](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)

![image.png](./assets/image2.png)

所以我们需要:
* 两个全连接层(fc1和fc2).
* 使用ReLU作为全连接层fc1的激活函数.
* 使用Softmax输出动作的概率分布.

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 创建两个全连接层.

    def forward(self, x):
        """定义前向传播."""
        # 状态输入到fc1, 然后使用ReLU激活.
        
        # fc1输出到fc2.
        
        # 最后使用softmax激活输出.

    def act(self, state):
        """给定一个状态获得一个动作."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state)
        m = Categorical(probs)
        action = np.argmax(m)

        return action.item(), m.log_prob(action)

#### 答案

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 创建两个全连接层.
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        """定义前向传播."""
        # 状态输入到fc1, 然后使用ReLU激活.
        x = F.relu(self.fc1(x))
        # fc1输出到fc2.
        x = self.fc2(x)
        # 最后使用softmax激活输出.
        return F.softmax(x, dim=1)

    def act(self, state):
        """给定一个状态获得一个动作."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state)
        m = Categorical(probs)
        action = np.argmax(m)

        return action.item(), m.log_prob(action)

我这里有一个错误, 你能猜到在哪里吗?
* 为了找到答案, 让我们前向传播:

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

* 这里我们看到错误说 `ValueError: The value argument to log_prob must be a Tensor`
* 这表明`m.log_prob(action)`中的`action`必须是一个张量, **但现在不是.**
* 你知道这是为什么吗? 检查函数`act`并尝试查看它为什么不正常工作.

建议 💡: 这个实现中有问题. 记住我们的函数`act`是**我们想从动作的概率分布中采样一个动作.**

#### 答案
* 由于**我们想从动作的概率分布中采样一个动作**, 但我们不能使用`action = np.argmax(m)`, 因为它总是输出具有最高概率的动作.
* 我们需要替换成`action = m.sample()`, 它将从概率分布$P(.|s)$中采样一个动作出来.

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 创建两个全连接层.
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        """定义前向传播."""
        # 状态输入到fc1, 然后使用ReLU激活.
        x = F.relu(self.fc1(x))
        # fc1输出到fc2.
        x = self.fc2(x)
        # 最后使用softmax激活输出.
        return F.softmax(x, dim=1)

    def act(self, state):
        """给定一个状态获得一个动作."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()

        return action.item(), m.log_prob(action)

通过使用CartPole, 调试起来更加容易, 因为**我们知道bug来自我们的代码而不是我们的环境.**

### 第5步: 构建Reinforce训练算法
* 与伪代码相反, 我们在每轮(episode)之后更新策略, **而不是使用一个批次的轮(batch of episodes).**

![image.png](./assets/image3.png)

为什么我们要最小化损失? 你说的是梯度上升(Gradient Ascent)而不是梯度下降(Gradient Descent)?
* 我们想最大化我们的目标函数$J(\theta)$, 但是在PyTorch中就像在TensorFlow中一样, **最好是最小化目标函数.**
    * 因此, 假设我们想在某个时间步加强动作$a_3$. 在训练这个动作之前, 它的概率是0.25.
    * 所以我们需要修改$\theta$, 使得$\pi_\theta(a_3|s;\theta)>0.25$
    * 因为概率和必须为1, 所以最大化$\pi_\theta(a_3|s;\theta)$将**最小化其他动作的概率.**
    * 所以我们应该告诉PyTorch去**最小化$1-\pi_\theta(a_3|s;\theta)$.**
    * 当$\pi_\theta(a_3|s;\theta)$趋近1时, 损失函数趋近0.
    * 所以这等同于最大化梯度$\pi_\theta(a_3|s;\theta)$

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # 帮助我们计算训练过程中的分数.
    score_deque = deque(maxlen=100)
    scores = []
    # 伪代码第3行.
    for i_episode in range(1, n_training_episodes + 1):
        saved_log_probs = []
        rewards = []
        state =  # TODO: 重置环境.
        # 伪代码第4行.
        for t in range(max_t):
            action, log_prob =  # TODO: 获取一个动作.
            saved_log_probs.append(log_prob)
            state, reward, done, info =  # TODO: 在环境上执行动作.
            rewards.append(reward)
            if done:
                break
        score_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # 伪代码第6行: 计算奖罚值.
        # 这里我们计算衰减, 例如: [0.99^1, 0.99^2, 0.99^3, ..., 0.99^len(rewards)]
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        # 我们计算奖罚值的总和sum(gamma[t] * reward[t])
        R = sum([a * b for a, b in zip( , )])  # TODO: 我们需要在函数zip()中填入什么, 请记住我们计算gamma[t] * reward[t]

        # 伪代码第7行.
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()

        # 伪代码第8行: PyTorch执行计算.
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('第{}轮\t 平均得分{:.2f}'.format(i_episode, np.mean(score_deque)))
            
    return scores

#### 答案

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # 帮助我们计算训练过程中的分数.
    score_deque = deque(maxlen=100)
    scores = []
    # 伪代码第3行.
    for i_episode in range(1, n_training_episodes + 1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # 伪代码第4行.
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, info = env.step(action)
            rewards.append(reward)
            if done:
                break
        score_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # 伪代码第6行: 计算奖罚值.
        # 这里我们计算衰减, 例如: [0.99^1, 0.99^2, 0.99^3, ..., 0.99^len(rewards)]
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        # 我们计算奖罚值的总和sum(gamma[t] * reward[t])
        R = sum([a * b for a, b in zip(discounts, rewards)])

        # 伪代码第7行.
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()

        # 伪代码第8行: PyTorch执行计算.
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('第{}轮\t 平均得分{:.2f}'.format(i_episode, np.mean(score_deque)))

    return scores

### 训练
* 我们已经准备好训练我们的智能体.
* 首先, 我们需要定义一个包含所有训练超参数的变量.

In [ ]:
cartpole_hyperparameters = {
    'h_size': 16,
    'n_training_episodes': 1000,
    'n_evaluation_episodes': 10,
    'max_t': 1000,
    'gamma': 1.0,
    'lr': 1e-2,
    'env_id': env_id,
    'state_space': s_size,
    'action_space': a_size
}

In [ ]:
# 创建模型并将它放到硬件设备上.
cartpole_policy = Policy(cartpole_hyperparameters['state_space'],
                         cartpole_hyperparameters['action_space'],
                         cartpole_hyperparameters['h_size']).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters['lr'])

In [ ]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters['n_training_episodes'],
                   cartpole_hyperparameters['max_t'],
                   cartpole_hyperparameters['gamma'],
                   100)

### 第6步:  定义评估函数 📝

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
    """用`n_eval_episodes`轮评估智能体, 并返回奖励的均值和标准差.

    Args:
        env: 评估环境.
        max_steps: 每轮的最大步数.
        n_eval_episodes: 测试的总轮数.
        policy: Reinforce智能体.

    Returns:
        奖励的均值和标准差.
    """
    episode_rewards = []

    for episode in range(n_eval_episodes):
        state = env.reset()
        total_rewards_ep = 0

        for step in range(max_steps):
            action, _ = policy.act(state)
            new_state, reward, done, info = env.step(action)
            total_rewards_ep += reward

            if done:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)
        
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)
    
    return mean_reward, std_reward

### 第7步: 评估我们的智能体 📈

In [ ]:
evaluate_agent(eval_env,
               cartpole_hyperparameters['max_t'],
               cartpole_hyperparameters['n_evaluation_episodes'],
               cartpole_policy)

### 第8步(不涉及核心内容, 可选): 发布我们训练好的模型到Hub上 🚀
现在我们看到经过训练之后得到了很棒的结果, 我们可以通过一行代码发布我们训练的模型到hub 🤗 上.

这有一个模型卡的例子:

![image.png](./assets/image4.png)

在底层, Hub使用基于git的仓库(即使你不知道什么是git也不用担心), 这意味着你可以在实验和提高你的智能体以后更新新版本的模型.

### 发布到Hugging Face Hub
#### 请勿修改下面的代码

In [ ]:
%%capture
import datetime
import json
from pathlib import Path

from huggingface_hub import HfApi, Repository
from huggingface_hub.repocard import metadata_eval_result, metadata_save

In [ ]:
def record_video(env, policy, out_directory, fps=30):
    images = []  
    done = False
    state = env.reset()
    img = env.render(mode='rgb_array')
    images.append(img)
    
    while not done:
        # 在给定状态下, 采取具有最大期望奖励的动作(索引).
        action, _ = policy.act(state)
        state, reward, done, info = env.step(action) # 我们直接使用next_state = state来记录顺序(recording logic).
        img = env.render(mode='rgb_array')
        images.append(img)
        
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
import os


def package_to_hub(repo_id,
                   model,
                   hyperparameters,
                   eval_env,
                   video_fps=30,
                   local_repo_path='hub',
                   commit_message='Push Reinforce agent to Hub',
                   token=None):
    _, repo_name = repo_id.split('/')

    # 第0步: 克隆或创建仓库.
    # 创建仓库(如果内容不为空, 则克隆).
    api = HfApi()

    repo_url = api.create_repo(repo_id=repo_id,
                               token=token,
                               private=False,
                               exist_ok=True)

    # git pull
    repo_local_path = Path(local_repo_path) / repo_name
    repo = Repository(repo_local_path, clone_from=repo_url, use_auth_token=True)
    repo.git_pull()

    repo.lfs_track(['*.mp4'])

    # 第1步: 保存模型.
    torch.save(model, os.path.join(repo_local_path, 'model.pt'))

    # 第2步: 保存超参数到JSON.
    with open(Path(repo_local_path) / 'hyperparameters.json', 'w') as outfile:
        json.dump(hyperparameters, outfile)

    # 第3步: 评估模型并构建JSON.
    mean_reward, std_reward = evaluate_agent(eval_env,
                                             hyperparameters['max_t'],
                                             hyperparameters['n_evaluation_episodes'],
                                             model)

    # 首先, 获取当前时间.
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
        'env_id': hyperparameters['env_id'],
        'mean_reward': mean_reward,
        'n_evaluation_episodes': hyperparameters['n_evaluation_episodes'],
        'eval_datetime': eval_form_datetime,
    }
    # 写入JSON文件.
    with open(Path(repo_local_path) / 'results.json', 'w') as outfile:
        json.dump(evaluate_data, outfile)

    # 第4步: 创建模型卡.
    env_name = hyperparameters['env_id']

    metadata = {
        'tags': [
            env_name,
            'reinforce',
            'reinforcement-learning',
            'custom-implementation',
            'deep-rl-class'
        ]
    }

    # 添加评估.
    eval = metadata_eval_result(model_pretty_name=repo_name,
                                task_pretty_name='reinforcement-learning',
                                task_id='reinforcement-learning',
                                metrics_pretty_name='mean_reward',
                                metrics_id='mean_reward',
                                metrics_value=f'{mean_reward:.2f} +/- {std_reward:.2f}',
                                dataset_pretty_name=env_name,
                                dataset_id=env_name)

    # 合并所有的字典{metadata, eval}.
    metadata = {**metadata, **eval}

    model_card = f'''
      # 使用**Reinforce**智能体来玩**{env_id}**
      这是一个使用**Reinforce**训练有素的模型玩**{env_id}**.
      要学习使用这个模型并训练你的模型, 请查阅深度强化学习课程第5单元: https://github.com/huggingface/deep-rl-class/tree/main/unit5
      '''

    readme_path = repo_local_path / 'README.md'

    if readme_path.exists():
        with readme_path.open('r', encoding='utf8') as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open('w', encoding='utf-8') as f:
        f.write(readme)

    # 保存我们的评估信息到README的元数据.
    metadata_save(readme_path, metadata)

    # 第5步: 录制回放视频.
    video_path = repo_local_path / 'replay.mp4'
    record_video(env, model, video_path, video_fps)

    # 发布到Hub.
    print(f'发布 {repo_name} 到你的Hugging Face Hub')
    repo.push_to_hub(commit_message=commit_message)

    print(f'你的模型已经发布到Hub. 你可以点击链接查看的你的模型: {repo_url}')

通过使用`package_to_hub`, **你可以评估, 记录回放视频, 生成智能体的模型卡并把它发布到hub.**

看这边:

* 你可以**展示我们的作品** 🔥
* 你可以**可视化智能体的活动** 👀
* 你可以**与社区分享其他人也可以使用的智能体** 💾
* 你可以**访问排行榜🏆以查看你的智能体和你同学的智能体相比如何** 👉 https://huggingface.co/spaces/chrisjay/Deep-Reinforcement-Learning-Leaderboard

为了能分享你的模型到社区, 有以下三个步骤需要做:

1⃣️ (如果没有完成)创建一个Hugging Face账户 ➡ https://huggingface.co/join

2⃣️ 登陆账户, 然后你需要保存一个Hugging Face的身份验证令牌(token).

* 创建一个新的具有**写入规则**的令牌(https://huggingface.co/settings/tokens)

![image.png](./assets/image5.png)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

如果你使用IDE, 也可在终端中使用以下命令:

In [ ]:
!huggingface-cli login

3⃣️ 我们现在准备好使用`package_to_hub()`发布我们训练的智能体到🤗 Hub 🔥.

In [ ]:
repo_id = ''  # TODO: 定义你的仓库id:{你的用户名/Reinforce-{model-id}}.
package_to_hub(repo_id,
               cartpole_policy,  # 我们想保存的模型.
               cartpole_hyperparameters,  # 超参数.
               eval_env,  # 评估环境.
               video_fps=30,
               local_repo_path='hub',
               commit_message='Push Reinforce agent to the Hub')

现在我们尝试我们实现的智能体的鲁棒性, 让我们实验一下更复杂的环境例如:
* PixelCopter
* Pong

## 第2个智能体: PixelCopter 🚁
### 第1步: 研究PixelCopter环境 👀
* [环境的文档](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

可观察的环境空间的形状(7,) 👀:
* 玩家y坐标
* 玩家的速度
* 玩家到地面的距离
* 玩家到顶部的距离
* 下一个方块x坐标到玩家的距离
* 下一个方块顶部的y坐标
* 下一个方块底部的y坐标

动作空间有2个可用动作 🎮:
* 向上移动,
* 向下移动.

奖励函数(在每个时间步给予的奖励)💰:
* 通过每个垂直方块, 它都会得到+1的奖励. 每次都到结束状态都得到-1的惩罚.

In [ ]:
env_id = 'Pixelcopter-PLE-v0'

env = gym.make(env_id)
eval_env = gym.make(env_id)

s_size = env.observation_space.shape[0]
a_size = env.action_space.n

### 第2步: 定义超参数 ⚙️
* 因为这个环境比较复杂, 所以我们需要调整超参数.
* 特别是隐藏层的大小, 我们需要更多的神经元.

In [ ]:
pixelcopter_hyperparameters = {
    'h_size': 64,
    'n_training_episodes': 50000,
    'n_evaluation_episodes': 10,
    'max_t': 10000,
    'gamma': 0.99,
    'lr': 1e-4,
    'env_id': env_id,
    'state_space': s_size,
    'action_space': a_size
}

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()

        return action.item(), m.log_prob(action)

In [ ]:
# 创建模型并将它放到硬件设备上.
# torch.manual_seed(50)
pixelcopter_policy = Policy(pixelcopter_hyperparameters['state_space'],
                            pixelcopter_hyperparameters['action_space'],
                            pixelcopter_hyperparameters['h_size']).to(device)
pixelcopter_optimizer = optim.Adam(pixelcopter_policy.parameters(), lr=pixelcopter_hyperparameters['lr'])

In [ ]:
scores = reinforce(pixelcopter_policy,
                   pixelcopter_optimizer,
                   pixelcopter_hyperparameters['n_training_episodes'],
                   pixelcopter_hyperparameters['max_t'],
                   pixelcopter_hyperparameters['gamma'],
                   1000)

In [ ]:
repo_id = ''  # TODO: 定义你的仓库id:{你的用户名/Reinforce-{model-id}}.
package_to_hub(repo_id,
               pixelcopter_policy,  # 我们想保存的模型.
               pixelcopter_hyperparameters,  # 超参数.
               eval_env,  # 评估环境.
               video_fps=30,
               local_repo_path='hub',
               commit_message='Push Reinforce agent to the Hub')

## 第3个智能体: Pong 🎾
### 第1步: 研究Pong环境 👀
* [环境的文档](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pong.html)

可观察的环境空间的形状(7,) 👀:
* 玩家y坐标
* 玩家的速度
* 电脑的位置
* 球的x坐标
* 球的y坐标
* 球的水平速度
* 球的垂直速度

动作空间有3个可用动作 🎮:
* 向上移动,
* 向下移动,
* 不移动.

奖励函数(在每个时间步给予的奖励)💰:
* 每个对手未接到的球(在球拍后), 它都会得到+1的奖励. 每个未接到的球都得到-1的惩罚.

In [ ]:
env_id = 'Pong-PLE-v0'

env = gym.make(env_id)
eval_env = gym.make(env_id)

s_size = env.observation_space.shape[0]
a_size = env.action_space.n

### 第2步: 定义超参数 ⚙️
* 因为这个环境比较复杂, 所以我们需要调整超参数.
* 特别是隐藏层的大小, 我们需要更多的神经元.

In [ ]:
pong_hyperparameters = {
    'h_size': 64,
    'n_training_episodes': 20000,
    'n_evaluation_episodes': 10,
    'max_t': 5000,
    'gamma': 0.99,
    'lr': 1e-2,
    'env_id': env_id,
    'state_space': s_size,
    'action_space': a_size
}

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size * 2)
        self.fc3 = nn.Linear(h_size * 2, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()

        return action.item(), m.log_prob(action)

### 第3步: 训练智能体 🏃

In [ ]:
# 创建模型并将它放到硬件设备上.
# torch.manual_seed(50)
pong_policy = Policy(pong_hyperparameters['state_space'],
                     pong_hyperparameters['action_space'],
                     pong_hyperparameters['h_size']).to(device)
pong_optimizer = optim.Adam(pong_policy.parameters(), lr=pong_hyperparameters['lr'])

In [ ]:
scores = reinforce(pong_policy,
                   pong_optimizer,
                   pong_hyperparameters['n_training_episodes'],
                   pong_hyperparameters['max_t'],
                   pong_hyperparameters['gamma'],
                   1000)

### 第4步: 发布我们训练好的模型到Hugging Face Hub 🔥

In [ ]:
repo_id = ''  # TODO: 定义你的仓库id:{你的用户名/Reinforce-{model-id}}.
package_to_hub(repo_id,
               pong_policy,  # 我们想保存的模型.
               pong_hyperparameters,  # 超参数.
               eval_env,  # 评估环境.
               video_fps=30,
               local_repo_path='hub',
               commit_message='Push Reinforce agent to the Hub')

## 额外的挑战(可选) 🏆
最好的学习方式就是**自己进行尝试**! 如你所见, 当前的智能体还有做到最好. 作为第一个建议, 你可以训练更多的时间步. 也可以尝试找到更好的参数.

在[排行榜](https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard)中, 你将找到你的智能体的位置. 你想要获得第一吗?

以下是一些实现这个目标的想法:
* 训练更多的时间步
* 尝试不同的超参数. 你可以在 👉 https://huggingface.co/models?other=reinforce 看到其他同学的超参数它们
* **发布你训练的新模型**到Hub上 🔥

---
祝贺你完成本章! 这才是最重要的, 这还有**一些额外的信息**.

如果你仍然对这些感到困惑...这是完全正常的! **这对我和所有学习强化学习的人都是一样的**.

在继续尝试其他挑战之前, **花一点时间真正的掌握这些内容**. 理解这些内容并打下基础是非常重要的.

当然, 在后续课程中, 我们将会继续使用并再次解释这些内容, 但**最好是在开始下一章之前完全掌握这些**.

### 这是专门为你打造的课程 👷🏿‍♀️

我们希望根据你的反馈提高和改进课程. 如果你有一些建议, 请打开GitHub仓库的issue: https://github.com/huggingface/deep-rl-class/issues

第6单元见! 🔥
## 不断学习, 不断精彩! 